In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lewidi_lib import (
    assign_cols_perf_metrics,
    enable_logging,
    join_correct_responses,
    load_preds,
    make_query_from_dict,
    process_rdf,
)
import pandas as pd

enable_logging()


ratings = pd.read_json(
    "../parquets/reasoning-ratings/template-2-reasoning-judge-responses.jsonl",
    lines=True,
)


rdf = load_preds(parquets_dir="../parquets")


def preprocess(rdf: pd.DataFrame, model_id="Qwen/Qwen3-32B") -> pd.DataFrame:
    metadata = {
        "template_id": 31,
        "model_id": model_id,
        "gen_kwargs": "set2",
        "dataset": "CSC",
        "judge_model_id": "gemini-2.5-pro",
    }
    query = make_query_from_dict(metadata, rdf.columns)
    rdf = rdf.query(query)
    rdf = process_rdf(rdf)
    rdf = join_correct_responses(rdf)
    rdf = assign_cols_perf_metrics(rdf)
    return rdf


rdf = preprocess(rdf)

In [ ]:
import json_repair
from lewidi_lib import drop_failed_rows, drop_na_response_rows
import numpy as np
from prm800k import extract_rating

ratings = drop_failed_rows(ratings)
ratings["step_ratings"] = (
    ratings["response"].apply(json_repair.loads).apply(extract_rating)
)
ratings = drop_na_response_rows(ratings, col="step_ratings")
ratings["avg_step_rating"] = ratings["step_ratings"].apply(np.mean)

In [40]:
join_cols = ["dataset", "dataset_idx", "run_idx"]  # expand when more cols!
ratings_cols = [
    "step_ratings",
    "avg_step_rating",
    "reasoning",
    "judge_model_id",
    "dataset",
    "dataset_idx",
    "run_idx",
]
joint = ratings[ratings_cols].merge(
    rdf, on=join_cols, how="inner", suffixes=("_judge", "")
)

In [ ]:
joint[["avg_step_rating", "ws_loss"]].corr()

##### example 8: too lax with spread out distribution
for k, v in (
    joint.iloc[37][
        ["text", "reasoning", "response", "target", "reasoning_judge", "response_judge"]
    ]
    .to_dict()
    .items()
):
    print(k, v)
    print("-" * 100)

In [ ]:
import seaborn as sns

# Using JointGrid directly for more control
fgrid = sns.JointGrid(data=joint, x="avg_step_rating", y="ws_loss")
fgrid.plot_joint(sns.scatterplot, alpha=0.5)
fgrid.plot_joint(sns.regplot, scatter=False)  # Add regression line
fgrid.plot_marginals(sns.histplot)

In [ ]:
(joint.groupby("dataset_idx").size() == 10).all()

In [ ]:
rdflarge = load_preds("/mnt/disk16tb/globus_shared/from-lrz-ai-systems/")
rdflarge = preprocess(rdflarge)

In [ ]:
# there is almost no performance difference between the normal outputs
# and those selected for top trace ratings
avg_ws_loss = rdflarge.groupby("dataset_idx", as_index=False).agg(
    ws_loss=("ws_loss", "mean"), pred_entropy=("pred_entropy", "mean")
)
avg_ws_loss.mean()

In [46]:
best_by_judge = joint.loc[joint.groupby("dataset_idx")["avg_step_rating"].idxmax()][
    [
        "dataset_idx",
        "avg_step_rating",
        "tgt_has_holes",
        "ws_loss",
        "pred_entropy",
        "target_entropy",
    ]
]

In [ ]:
from lewidi_lib import (
    agg_perf_metrics,
    compute_average_baseline,
    process_rdf_and_add_perf_metrics,
)

model_avg_baseline = compute_average_baseline(rdflarge)
gemini_raw = (
    load_preds("../parquets/baseline")
    .query("template_id == 31")
    .pipe(process_rdf_and_add_perf_metrics)
)
gemini_agg = agg_perf_metrics(gemini_raw)
gemini_model_avg = agg_perf_metrics(compute_average_baseline(gemini_raw))

In [ ]:
rdflarge = rdflarge.assign(
    entropy_rank=rdflarge.groupby("dataset_idx")["pred_entropy"]
    .rank(method="first")
    .astype(int)
)
by_entropy = rdflarge.groupby("entropy_rank", as_index=False)[
    ["ws_loss", "pred_entropy"]
].mean()
by_entropy["type"] = (
    "entropy"  # "entropy r" + (by_entropy["entropy_rank"] - 1).astype(str)
)
by_entropy.head(2)

In [71]:
cols = ["pred_entropy", "ws_loss"]
loss_vs_entropy = pd.DataFrame(
    {
        "best_by_judge": best_by_judge[cols].mean(),
        "simple": avg_ws_loss[cols].mean(),
        "model_avg_baseline": model_avg_baseline[cols].mean(),
        "gemini-2.5-pro": gemini_agg[cols].mean(),
        "gemini-2.5-pro-model-avg": gemini_model_avg[cols].mean(),
    }
).T.reset_index(names="type")
loss_vs_entropy = pd.concat([loss_vs_entropy, by_entropy.drop(columns="entropy_rank")])

In [ ]:
sns.set_context("talk")
grid = sns.JointGrid(data=loss_vs_entropy, x="pred_entropy", y="ws_loss")
grid.plot_joint(sns.scatterplot, hue=loss_vs_entropy["type"], style=loss_vs_entropy["type"])
grid.plot_marginals(sns.histplot, multiple="stack")
grid.ax_joint.legend(bbox_to_anchor=(1.2, 1), loc="upper left")
grid.ax_joint.grid(alpha=0.5)

import matplotlib.pyplot as plt
import seaborn as sns

grid1 = sns.jointplot(data=model_avg_baseline, x="pred_entropy", y="ws_loss", marginal_kws={'bins': 20})
grid1.fig.suptitle("Model Avg Baseline", y=1.02)
ylim = (rdflarge["ws_loss"].min(), rdflarge["ws_loss"].max())
grid1.ax_joint.set_ylim(ylim)

grid2 = sns.jointplot(data=rdflarge, x="pred_entropy", y="ws_loss", marginal_kws={'bins': 20})
grid2.fig.suptitle("Simple", y=1.02)
grid1.ax_joint.set_ylim(ylim)

In [ ]:
hist_data = pd.concat(
    [
        rdflarge.assign(type="simple"),
        model_avg_baseline.assign(type="model_avg_baseline"),
        best_by_judge.assign(type="best_by_judge"),
    ]
)

sns.violinplot(hist_data, x="type", y="ws_loss", common_norm=False)

# Tail of High Loss

In [ ]:
best_by_judge.sort_values("ws_loss", ascending=False).head(3)

In [ ]:
sns.lmplot(best_by_judge, x="pred_entropy", y="ws_loss")

In [ ]:
sns.scatterplot(
    joint.query("dataset_idx == 6827"),
    x="avg_step_rating",
    y="ws_loss",
    hue="pred_entropy",
)

In [55]:
max_ratings = joint.groupby("dataset_idx", as_index=False).agg(
    n_max_ratings=("avg_step_rating", lambda s: (s.max() == s).sum()),
    max_rating=("avg_step_rating", "max"),
    avg_pred_entropy=("pred_entropy", "mean"),
)

In [ ]:
sns.lmplot(max_ratings.merge(best_by_judge), x="n_max_ratings", y="avg_pred_entropy")

# When does Model Averaging improve?

In [ ]:
improvements = (
    rdflarge.groupby(["dataset_idx", "tgt_has_holes"], as_index=False)
    .agg(
        avg_ws_loss=("ws_loss", "mean"),
        avg_pred_entropy=("pred_entropy", "mean"),
    )
    .merge(model_avg_baseline[["dataset_idx", "ws_loss", "pred_entropy"]])
    .assign(improvement=lambda df: df["avg_ws_loss"] - df["ws_loss"])
    .assign(pred_entropy_diff=lambda df: df["pred_entropy"] - df["avg_pred_entropy"])
)

improvements.sort_values("improvement", ascending=False).head(3)

In [ ]:
sns.scatterplot(
    improvements,
    x="improvement",
    y="pred_entropy_diff",
)

In [ ]:
sns.histplot(improvements, x="improvement", hue="tgt_has_holes")

In [ ]:
example = rdflarge.query("dataset_idx == 2092").sort_values("ws_loss")
example[["pred", "ws_loss"]].tail(5)

In [ ]:
example["ws_loss"].mean()

In [ ]:
example_improved = compute_average_baseline(example)
print(example_improved["ws_loss"])
sns.barplot(example_improved["target"].iloc[0])